In [1]:
# Traffic & Air Quality Analysis Using datascience + GeoPandas + DBSCAN + Geohashing

# STEP 1: Import libraries
# ----------------------------------------
# Import all necessary libraries for data handling, spatial analysis, plotting, and clustering
from datascience import Table
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import geohash2
import glob

In [2]:
aq_df = pd.read_csv("spatial_join_data.csv")

In [3]:
aq_df

,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Centroid Latitude,Pickup Centroid Longitude,trip_minutes,avg_speed_mph,geometry,index_right,Latitude,Longitude,ReadingDateTimeUTC,PM25,CalibratedPM25,CalibratedO3,CalibratedNO2,CO
0,2021-04-23 21:15:00+00:00,2021-04-23 21:15:00+00:00,0.0,0.00,41.714270,-87.560057,0.000000,NaN,POINT (-9747140.987567797 5118274.180775281),68269,41.713600,-87.560200,2021-07-01 13:35:55+00:00,11.961367,23.49,31.41,16.40,10.894218
1,2021-04-23 21:15:00+00:00,2021-04-23 21:15:00+00:00,0.0,0.00,41.714270,-87.560057,0.000000,NaN,POINT (-9747140.987567797 5118274.180775281),68270,41.713600,-87.560200,2021-07-01 13:41:05+00:00,289.035583,209.31,31.99,14.81,9.078482
2,2021-04-23 21:15:00+00:00,2021-04-23 21:15:00+00:00,0.0,0.00,41.714270,-87.560057,0.000000,NaN,POINT (-9747140.987567797 5118274.180775281),68271,41.713600,-87.560200,2021-07-01 13:46:16+00:00,8.686007,20.90,29.85,14.74,8.803521
3,2021-04-23 21:15:00+00:00,2021-04-23 21:15:00+00:00,0.0,0.00,41.714270,-87.560057,0.000000,NaN,POINT (-9747140.987567797 5118274.180775281),68272,41.713600,-87.560200,2021-07-01 13:51:26+00:00,8.745945,20.90,30.87,14.96,4.501060
4,2021-04-23 21:15:00+00:00,2021-04-23 21:15:00+00:00,0.0,0.00,41.714270,-87.560057,0.000000,NaN,POINT (-9747140.987567797 5118274.180775281),68273,41.713600,-87.560200,2021-07-01 13:56:36+00:00,5.722027,17.04,26.82,14.80,2.289102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217297284,2021-08-05 11:45:00+00:00,2021-08-05 12:00:00+00:00,1020.0,0.00,41.985472,-87.653794,17.000000,0.000000,POINT (-9757575.661747018 5158803.531669278),1414864,41.985095,-87.655334,2021-10-09 23:59:05+00:00,2.989588,5.22,35.59,8.66,0.078180
217297285,2021-06-12 15:30:00+00:00,2021-06-12 16:45:00+00:00,4342.0,34.41,41.985472,-87.653794,72.366667,28.529710,POINT (-9757575.661747018 5158803.531669278),1414864,41.985095,-87.655334,2021-10-09 23:59:05+00:00,2.989588,5.22,35.59,8.66,0.078180
217297286,2021-01-15 08:45:00+00:00,2021-01-15 09:15:00+00:00,1159.0,8.42,41.985472,-87.653794,19.316667,26.153581,POINT (-9757575.661747018 5158803.531669278),1414864,41.985095,-87.655334,2021-10-09 23:59:05+00:00,2.989588,5.22,35.59,8.66,0.078180
217297287,2021-05-07 22:45:00+00:00,2021-05-07 22:45:00+00:00,2.0,0.00,41.985472,-87.653794,0.033333,0.000000,POINT (-9757575.661747018 5158803.531669278),1414864,41.985095,-87.655334,2021-10-09 23:59:05+00:00,2.989588,5.22,35.59,8.66,0.078180
